In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.

In [8]:
import machine
spi = machine.SPI(1, baudrate=100000, polarity=0, phase=0)


In [11]:
spi.write(b'\x76')
spi.write(b'\x77\ff');

 

In [ ]:

// Define the SS pin
//  This is the only pin we can move around to any available
//  digital pin.
const int ssPin = 8;

unsigned int counter = 0;  // This variable will count up to 65k
char tempString[10];  // Will be used with sprintf to create strings

void setup()
{
  // -------- SPI initialization
  pinMode(ssPin, OUTPUT);  // Set the SS pin as an output
  digitalWrite(ssPin, HIGH);  // Set the SS pin HIGH
  SPI.begin();  // Begin SPI hardware
  SPI.setClockDivider(SPI_CLOCK_DIV64);  // Slow down SPI clock
  // --------

  // Clear the display, and then turn on all segments and decimals
  clearDisplaySPI();  // Clears display, resets cursor

  // Custom function to send four bytes via SPI
  //  The SPI.transfer function only allows sending of a single
  //  byte at a time.
  s7sSendStringSPI("-HI-");
  setDecimalsSPI(0b111111);  // Turn on all decimals, colon, apos

  // Flash brightness values at the beginning
  setBrightnessSPI(0);  // Lowest brightness
  delay(1500);
  setBrightnessSPI(255);  // High brightness
  delay(1500);

  // Clear the display before jumping into loop
  clearDisplaySPI();  
}

void loop()
{
  // Magical sprintf creates a string for us to send to the s7s.
  //  The %4d option creates a 4-digit integer.
  sprintf(tempString, "%4d", counter);

  // This will output the tempString to the S7S
  s7sSendStringSPI(tempString);

  // Print the decimal at the proper spot
  if (counter < 10000)
    setDecimalsSPI(0b00000010);  // Sets digit 3 decimal on
  else
    setDecimalsSPI(0b00000100);

  counter++;  // Increment the counter
  delay(10);  // This will make the display update at 100Hz.*/
}

// This custom function works somewhat like a serial.print.
//  You can send it an array of chars (string) and it'll print
//  the first 4 characters in the array.
void s7sSendStringSPI(String toSend)
{
  digitalWrite(ssPin, LOW);
  for (int i=0; i<4; i++)
  {
    SPI.transfer(toSend[i]);
  }
  digitalWrite(ssPin, HIGH);
}

// Send the clear display command (0x76)
//  This will clear the display and reset the cursor
void clearDisplaySPI()
{
  digitalWrite(ssPin, LOW);
  SPI.transfer(0x76);  // Clear display command
  digitalWrite(ssPin, HIGH);
}

// Set the displays brightness. Should receive byte with the value
//  to set the brightness to
//  dimmest------------->brightest
//     0--------127--------255
void setBrightnessSPI(byte value)
{
  digitalWrite(ssPin, LOW);
  SPI.transfer(0x7A);  // Set brightness command byte
  SPI.transfer(value);  // brightness data byte
  digitalWrite(ssPin, HIGH);
}

// Turn on any, none, or all of the decimals.
//  The six lowest bits in the decimals parameter sets a decimal 
//  (or colon, or apostrophe) on or off. A 1 indicates on, 0 off.
//  [MSB] (X)(X)(Apos)(Colon)(Digit 4)(Digit 3)(Digit2)(Digit1)
void setDecimalsSPI(byte decimals)
{
  digitalWrite(ssPin, LOW);
  SPI.transfer(0x77);
  SPI.transfer(decimals);
  digitalWrite(ssPin, HIGH);
}


In [13]:
%sendtofile --execute devices.py

import time, ustruct, ure, math, sys
import machine

# NodeMCU: (D1=SCL, D2=SDA)
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4), freq=20000)


Sent 5 lines (151 bytes).

In [14]:
%sendtofile --execute --append devices.py

scannedi2c = None
bmp180consts = None

def initdevices():
    global scannedi2c, bmp180consts
    
    scannedi2c = i2c.scan()
    res = [ ]

    if 0x77 in scannedi2c:
        res.append("0x77 is BMP180 barometer")
        bcd = i2c.readfrom_mem(0x77, 0xAA, 22)
        bmp180consts = dict(zip(("AC1", "AC2", "AC3", "AC4", "AC5", "AC6", "B1", "B2", "MB", "MC", "MD"), ustruct.unpack(">hhhHHHhhhhh", bcd)))  # note the 3 unsigned constants
        
    if 0x6B in scannedi2c:
        res.append("0x6B is Gyros/Accelerometer")
        
        # turn on gyros reg(0x10)0x20=(ODR_G,FS_G,0,BWG)=001 00 0 00 gives 65ms ~ 14.9Hz
        # turn on gyros reg(0x10)0x40=(ODR_G,FS_G,0,BWG)=010 00 0 00 gives 59.9Hz at 245deg/sec
        # turn on gyros reg(0x10)0x40=(ODR_G,FS_G,0,BWG)=010 11 0 00 gives 59.9Hz at 2000deg/sec
        i2c.writeto(0x6B, b'\x10\x40')  # these gyros don't seem to work anyway.  give constant values that respond to orienting (0x13), scaling(0x10), enabling(0x1e), but not any any measuring of the environment

        # turn on accelerometer reg(0x20)=(ODR_XL,FS_XL,BW_SCAL_ODR,BW_XL)=110 00 0 00 should give 952Hz, but is overridden by gyros ODR 
        # there are various further settings of FIFO and High and Low pass filters
        i2c.writeto(0x6B, b'\x20\xC0')  

    if 0x1E in scannedi2c:
        res.append("0x1E is Magnetometer(compass)")
        i2c.writeto(0x1E, b'\x22\x00')  

        # turn on magnetometer reg(0x20)=(TEMP_COMP,OM,DO,0,ST)=1 11 110 0 0 (Ultra High Performance, 40Hz)
        #i2c.writeto(0x1E, b'\x20\x58')  
        i2c.writeto(0x1E, b'\x20\x58')  
        
    if 0x69 in scannedi2c:
        res.append("0x69 is Figaro CO2 meter")
        i2c.writeto(0x69, b'\x01\x02')

    if 0x40 in scannedi2c:
        res.append("0x40 is SI7021 humidity sensor")
        i2c.writeto(0x40, b'\xFE')  # resets chip
        time.sleep(0.2)
        
    if 0x48 in scannedi2c:
        res.append("0x48 is TMP102 temperature sensor")

    if 0x28 in scannedi2c:
        k = i2c.readfrom_mem(0x28, 0x00, 6)
        res.append("BNO055 sensor SW_REV_ID: %s.%s" %(hex(k[4]), hex(k[5])))
        
        i2c.writeto_mem(0x28, 0x3D, b'\x00')     # config mode
        i2c.writeto_mem(0x28, 0x3E, b'\x00')     # PWR_MODE, normal
        i2c.writeto_mem(0x28, 0x3B, b'\x00')     # UNIT_SEL, celsius, UDegrees and m/s^2
        i2c.writeto_mem(0x28, 0x3D, b'\x0c')     # back to NDOF mode

    for a in scannedi2c:
        if a not in [0x68, 0x77, 0x32, 0x69, 0x6B, 0x1E, 0x48, 0x40, 0x70, 0x28]:
            res.append("%s: Unknown I2C device" % hex(a))
            
    return res


Sent 60 lines (2616 bytes).

In [5]:
print(initdevices())

['0x40 is SI7021 humidity sensor']


In [15]:
%sendtofile --execute --append devices.py

def SI7021checkchip():
    i2c.writeto(0x40, b'\xFA\x0F')
    sna = i2c.readfrom(0x40, 8)
    i2c.writeto(0x40, b'\xFC\xC9')
    snb = i2c.readfrom(0x40, 6)
    i2c.writeto(0x40, b'\x84\xB8')
    firmr = i2c.readfrom(0x40, 1)
    print("SNA %s %s %s %s  SNB %s %s %s %s  firmware %s" % (hex(sna[0]), hex(sna[2]), hex(sna[4]), hex(sna[6]), hex(snb[0]), hex(snb[1]), hex(snb[3]), hex(snb[4]), hex(firmr[0])))
    return (snb[0] == 21) # identifies the Si7021 type chip

def SI7021printstatus():
    reg1 = i2c.readfrom_mem(0x40, 0xE7, 1)[0]
    heater = i2c.readfrom_mem(0x40, 0x11, 1)[0]
    print("MeasRes:%s VDD:%s heater-on:%s heater:%s" % (hex(reg1 & 0x81), hex(reg1 & 0x40), hex(reg1 & 0x04), hex(heater & 0x0F)))

def SI7021setheater(hheater):
    # hheater to be between 0 and 15
    reg1 = i2c.readfrom_mem(0x40, 0xE7, 1)[0]
    nreg1 = (reg1 & 0xFB) if (hheater == 0) else (reg1 | 0x04) 
    i2c.writeto_mem(0x40, 0xE6, bytes([nreg1]))
    
    heater = i2c.readfrom_mem(0x40, 0x11, 1)[0]
    nheater = ((heater & 0xF0) | hheater); 
    i2c.writeto_mem(0x40, 0x51, bytes([nheater]))


def SI7021humiditytemp():
    i2c.writeto(0x40, b'\xF5')  # clock stretching hold type E5 seems not to work in micropython
    time.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    rh = ustruct.unpack(">H", bh)[0] & 0xFFFC
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rt = ustruct.unpack(">H", bt)[0] & 0xFFFC
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

def DewpointTemperature(humid, temp):
    A, B, C = 8.1332, 1762.39, 235.66
    svp = 10**(A - B/(temp + C))*133.322387415
    pvp = svp*humid/100
    return -C - B/(math.log10(pvp/133.322387415) - A)



Sent 41 lines (1712 bytes).

In [12]:
import ustruct
for i in range(10):
    print(SI7021humiditytemp())

(61.12341, 24.50576)
(61.07001, 24.51646)
(61.06238, 24.50576)
(61.0929, 24.50576)
(61.11578, 24.51646)
(61.12341, 24.52715)
(61.12341, 24.51646)
(61.13867, 24.52715)
(61.13867, 24.52715)
(61.17682, 24.52715)
